<a href="https://colab.research.google.com/github/khushiiagrawal/Scenario_1/blob/main/pfd_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain faiss-cpu openai PyMuPDF


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 25.5 MB/s eta 0:00:00


In [ ]:
pip install -U langchain langchain-openai langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [ ]:
import os
import fitz  # PyMuPDF
from dotenv import load_dotenv

from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

# Load API key from .env or directly
load_dotenv()
# Or uncomment below line and paste your key
os.environ["OPENAI_API_KEY"] = "you_api_key"


In [ ]:
# Step 1: Read PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    return "".join(page.get_text() for page in doc)

text = extract_text_from_pdf("/content/UP.pdf")

# Step 2: Chunk the text
splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=100)
documents = splitter.create_documents([text])

In [ ]:
# Step 3: Create embeddings using OpenAI
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vector_db = FAISS.from_documents(documents, embeddings)

# Step 4: Setup retriever + GPT-4o Mini LLM
retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 4})
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)


In [ ]:

# Step 5: Ask a question
query = "Summarize the document."
answer = qa_chain.run(query)
print("Answer:\n", answer)

/tmp/ipython-input-6-1832897116.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(query)


Answer:
 The document outlines the Uttar Pradesh Industrial Investment and Employment Promotion Policy 2022, which aims to establish Uttar Pradesh as a competitive investment destination, generate employment, and promote sustainable economic growth. The policy is structured around several key pillars, including infrastructure development, access to finance, skilled manpower, sectoral approaches, global value chain integration, investment attractiveness, and sustainability.

Key highlights of the policy include:

1. **Infrastructure Development**: Focus on creating quality infrastructure, including land banks, industrial parks, and improved connectivity (road, rail, air, and waterways).

2. **Access to Finance**: Initiatives to enhance financial support for industries, including fiscal incentives and the establishment of a FinTech City.

3. **Skilled Manpower**: Emphasis on skill development to align workforce capabilities with industry needs, including training programs and partnership